In [ ]:
from sklearn.externals import joblib
import numpy as np
from keras.models import Sequential,load_model
from keras.layers import LSTM, GRU, Dense, Activation, Dropout ,regularizers
from keras import optimizers
from sklearn.preprocessing import  StandardScaler
import pandas as pd

In [ ]:
train_df = pd.read_csv('./data/feature/train.csv',parse_dates=['date','time_start'],dtype={'link_ID':str})
train_df = train_df.sort_values(by=['link_ID','time_start'])
train_df = train_df[train_df.date>=pd.to_datetime('20170302',format='%Y%m%d')]
train_df.fillna(method='ffill',inplace=True)
train_df.dropna(axis=0,inplace=True)
val_data = train_df[(train_df.time_start_hour==8)&(train_df.month==5)]
val_index = val_data.index
test_data = train_df[(train_df.month==6)&(train_df.time_start_hour==8)]
test_index = test_data.index
rm_index = val_index.append(test_index)
rm_index = train_df.index.isin(rm_index)
train_data = train_df[~rm_index]
train_data = train_data[train_df.time_start<pd.Timestamp('2017-05-01-00')]
test_id = test_data.link_ID.values
test_time = test_data.time_start.values
col_list = train_data.drop(['link_ID','date','time_start','travel_time'],axis=1).columns.tolist()
train_y = train_data.travel_time.values
val_y = val_data.travel_time.values
train_data = train_data.drop(['link_ID','date','time_start','travel_time'],axis=1).values
val_data = val_data.drop(['link_ID','date','time_start','travel_time'],axis=1).values
dtest = test_data.drop(['link_ID','date','time_start','travel_time'],axis=1).values

In [ ]:
for i in range(train_data.shape[1]):
    sc = StandardScaler()
    sc.fit(train_data[:,i])
    train_data[:,i] = sc.transform(train_data[:,i].flatten())
    #test_data[:,i] = sc.transform(test_data[:,i])
    val_data[:,i] = sc.transform(val_data[:,i].flatten())

In [ ]:
def build_model(timesteps):
    model = Sequential()
        
    model.add(Dense(input_dim=timesteps,activation='relu',
                    output_dim=64))
    model.add(Dropout(0.3))
    
    model.add(Dense(128,activation='relu'))
    model.add(Dropout(0.3))
    
    model.add(Dense(192,activation='relu'))
    model.add(Dropout(0.4))
    

    
    model.add(Dense(output_dim=1))
    model.add(Activation("relu"))
    sgd = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    model.compile(loss='mean_absolute_percentage_error',
                  optimizer=sgd,
                  metrics=['mape'])
    return model

In [ ]:
model = build_model(train_data.shape[1])

In [ ]:
model.fit(batch_size=512,epochs=10,shuffle=True,x=train_data,y=train_y,validation_data=(val_data,val_y),verbose=1)

In [ ]:
def mape(pred,actual):
    return np.fabs(pred/actual-1).mean()

In [ ]:
pred = model.predict(val_data)

In [ ]:
pred = pred.flatten()

In [ ]:
mape(pred,val_y)